# Tugas Pertemuan 1


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://sport.detik.com/indeks"]
categories = ["Olahraga"]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    for page in range(1, 4):  # Looping untuk beralih halaman
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            break

        url = f"{base_url}/{page}"
        get_data(url, category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({
    "judul": judul,
    "tanggal": tanggal,
    "isi": isi
})

# Menyimpan dataframe ke file CSV
df.to_csv("Crawl-berita.csv", index=False)


In [12]:
df=pd.read_csv("Crawl-berita.csv")
df

,judul,tanggal,isi
0,Paralimpiade 2024: Qonitah Gagal Sumbang Emas ...,"Senin, 02 Sep 2024 14:18 WIB","Jakarta - Atlet Indonesia, Qonitah Ikhtiar Sya..."
1,Bukan Maksud Alex Marquez Lindas Bagnaia,"Senin, 02 Sep 2024 13:45 WIB",Jakarta - Tabrakan Alex Marquez dengan Frances...
2,"Mansion Sports Gelar Turnamen Biliar, Sediakan...","Senin, 02 Sep 2024 12:44 WIB","Jakarta - Para pecinta biliar, bersiaplah untu..."
3,Komentar Marc Marquez soal Tabrakan Adiknya de...,"Senin, 02 Sep 2024 12:30 WIB",Jakarta - Marc Marquez mengomentari tabrakan a...
4,"Lanjut Puasa Kemenangan, Verstappen: Tak Reali...","Senin, 02 Sep 2024 10:30 WIB",Monza - Pebalap Red Bull Racing Max Verstappen...
5,Duet Leo/Bagas Terus Melesat!,"Senin, 02 Sep 2024 07:58 WIB",Jakarta - Leo Rolly Carnando/Bagas Maulana mer...
6,Momen Alex Marquez Kesal Seusai Bertabrakan de...,"35,768 Views | Senin, 02 Sep 2024 04:33 WIB",Alex Marquez dan Francesco Bagnaia bertabrakan...
7,Paralimpiade 2024: Boccia Sumbang Tiga Medali ...,"Senin, 02 Sep 2024 05:54 WIB",Paris - Indonesia menambah tiga medali pada ha...
8,"Juara MotoGP Aragon, Marquez: Saya Mengorbanka...","8,949 Views | Senin, 02 Sep 2024 03:27 WIB",Marc Marquez berhasil menjadi juara MotoGP Ara...
9,Detik-Detik Marc Marquez Juarai MotoGP Aragon,"Senin, 02 Sep 2024 04:21 WIB",Jakarta - Marc Marquez menjuarai MotoGP Aragon...
